**This is the notebook for the development of the tool to help people to decide when to run theit appliances in order to help reduce grid management costs and achieve decarbonisation targets**

In [9]:
import os
from dotenv import load_dotenv

load_dotenv() # loads .env into shell

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY', None) # retrieves the API key from environment variable
ELEC_MAP_API_KEY = os.environ.get('ELEC_MAP_API_KEY', None)

In [10]:
import requests
import datetime
now = datetime.datetime.now(datetime.timezone.utc)
datetime_str_raw = now.strftime("%Y-%m-%dT%H:%M:%S.000Z").replace(":", "%3A")
print("Current date and time: ")
print(now)
print(datetime_str_raw )

Current date and time: 
2025-12-08 20:52:59.328237+00:00
2025-12-08T20%3A52%3A59.000Z


In [11]:
# Lets set up functions to get carbon intensity data

def get_carbon_intensity(zone, datetime_str):
    response = requests.get(
        f"https://api.electricitymaps.com/v3/carbon-intensity/past?zone={zone}&datetime={datetime_str}&disableEstimations=false&emissionFactorType=direct",
        headers={
            "auth-token": ELEC_MAP_API_KEY
        }
    )
    return response.json()


def get_carbon_intencity_forecast(zone):
    response = requests.get(
        f"https://api.electricitymaps.com/v3/carbon-intensity/forecast?zone={zone}&horizonHours=24",
        headers={
            "auth-token": ELEC_MAP_API_KEY
        }
    )
    return response.json()



In [12]:
# Define carbon intensity ranges (in gCO2eq/kWh) and functions to find best windows

carbon_intencity_range_low = 200  # gCO2eq/kWh
carbon_intencity_range_high = 400  # gCO2eq/kWh


def get_best_carbon_windows(forecast):
    """
    Find the best 2-hour windows for lowest carbon intensity.
    Returns a dict of suitable hours and wether those hours during the daytime or not.
    """
    # Best overall window
    best_idx = min(range(len(forecast) - 1),
                   key=lambda i: forecast[i]['carbonIntensity'] + forecast[i + 1]['carbonIntensity'])

    hour1 = forecast[best_idx]['datetime'][11:13]
    hour2 = forecast[best_idx + 1]['datetime'][11:13]
    total_intensity = forecast[best_idx]['carbonIntensity'] + forecast[best_idx + 1]['carbonIntensity']

    result = {
        "best_overall": {
            "start_hour": f"{hour1}:00",
            "end_hour": f"{hour2}:00",
            "total_intensity": total_intensity
        },
        "best_daytime": None,
        "is_best_during_daytime": 7 <= int(hour1) < 22
    }

    # If best overall is nighttime, find best daytime window
    if not result["is_best_during_daytime"]:
        daytime_indices = [i for i in range(len(forecast) - 1)
                          if 7 <= int(forecast[i]['datetime'][11:13]) < 22
                          and 7 <= int(forecast[i + 1]['datetime'][11:13]) < 22]

        if daytime_indices:
            best_day_idx = min(daytime_indices,
                               key=lambda i: forecast[i]['carbonIntensity'] + forecast[i + 1]['carbonIntensity'])

            day_hour1 = forecast[best_day_idx]['datetime'][11:13]
            day_hour2 = forecast[best_day_idx + 1]['datetime'][11:13]
            day_total = forecast[best_day_idx]['carbonIntensity'] + forecast[best_day_idx + 1]['carbonIntensity']

            result["best_daytime"] = {
                "start_hour": f"{day_hour1}:00",
                "end_hour": f"{day_hour2}:00",
                "total_intensity": day_total
            }

    return result


In [13]:
forecast_data = get_carbon_intencity_forecast('IE')
carbon_intensity = get_carbon_intensity('IE', datetime_str_raw)
print(carbon_intensity)
print(forecast_data)
print(get_best_carbon_windows(forecast_data['forecast']))

{'zone': 'IE', 'carbonIntensity': 235, 'datetime': '2025-12-08T20:00:00.000Z', 'updatedAt': '2025-12-08T20:40:16.001Z', 'createdAt': '2025-12-07T00:25:37.287Z', 'emissionFactorType': 'direct', 'isEstimated': True, 'estimationMethod': 'TEST_MODE_DATA', 'temporalGranularity': 'hourly', '_disclaimer': 'TEST MODE: intentionally inaccurate data for integration testing. For live data, head over to https://app.electricitymaps.com/ and request access.'}
{'zone': 'IE', 'forecast': [{'carbonIntensity': 320, 'datetime': '2025-12-08T20:00:00.000Z'}, {'carbonIntensity': 304, 'datetime': '2025-12-08T21:00:00.000Z'}, {'carbonIntensity': 340, 'datetime': '2025-12-08T22:00:00.000Z'}, {'carbonIntensity': 279, 'datetime': '2025-12-08T23:00:00.000Z'}, {'carbonIntensity': 211, 'datetime': '2025-12-09T00:00:00.000Z'}, {'carbonIntensity': 205, 'datetime': '2025-12-09T01:00:00.000Z'}, {'carbonIntensity': 194, 'datetime': '2025-12-09T02:00:00.000Z'}, {'carbonIntensity': 235, 'datetime': '2025-12-09T03:00:00.00

In [26]:
from google.adk import Agent
from google.adk.tools import FunctionTool
from google.adk.runners import InMemoryRunner
from google.genai import types

def get_best_usage_windows(zone: str) -> dict:
    """Find best time windows for electricity usage based on carbon intensity with emission comparisons."""
    forecast_data = get_carbon_intencity_forecast(zone)
    if "forecast" not in forecast_data:
        return {"error": "Could not fetch forecast"}
    
    forecast = forecast_data["forecast"]
    current_intensity = forecast[0]["carbonIntensity"] if forecast else None
    
    best_idx = min(range(len(forecast) - 1),
                   key=lambda i: forecast[i]['carbonIntensity'] + forecast[i + 1]['carbonIntensity'])
    
    hour1 = forecast[best_idx]['datetime'][11:13]
    hour2 = forecast[best_idx + 1]['datetime'][11:13]
    best_intensity = (forecast[best_idx]['carbonIntensity'] + forecast[best_idx + 1]['carbonIntensity']) / 2
    
    result = {
        "current": {
            "carbon_intensity_gCO2_per_kWh": current_intensity,
            "datetime": forecast[0]["datetime"] if forecast else None
        },
        "best_overall": {
            "start_hour": f"{hour1}:00",
            "end_hour": f"{hour2}:00",
            "carbon_intensity_gCO2_per_kWh": best_intensity,
            "savings_vs_now_percent": round((1 - best_intensity / current_intensity) * 100, 1) if current_intensity else None
        },
        "best_daytime": None,
        "is_best_during_daytime": 7 <= int(hour1) < 22
    }
    
    if not result["is_best_during_daytime"]:
        daytime_indices = [i for i in range(len(forecast) - 1)
                          if 7 <= int(forecast[i]['datetime'][11:13]) < 22
                          and 7 <= int(forecast[i + 1]['datetime'][11:13]) < 22]
        
        if daytime_indices:
            best_day_idx = min(daytime_indices,
                               key=lambda i: forecast[i]['carbonIntensity'] + forecast[i + 1]['carbonIntensity'])
            
            day_hour1 = forecast[best_day_idx]['datetime'][11:13]
            day_hour2 = forecast[best_day_idx + 1]['datetime'][11:13]
            day_intensity = (forecast[best_day_idx]['carbonIntensity'] + forecast[best_day_idx + 1]['carbonIntensity']) / 2
            
            result["best_daytime"] = {
                "start_hour": f"{day_hour1}:00",
                "end_hour": f"{day_hour2}:00",
                "carbon_intensity_gCO2_per_kWh": day_intensity,
                "savings_vs_now_percent": round((1 - day_intensity / current_intensity) * 100, 1) if current_intensity else None
            }
    
    return result

carbon_intensity_tool = FunctionTool(func=get_carbon_intensity)
carbon_forecast_tool = FunctionTool(func=get_carbon_intencity_forecast)
best_windows_tool = FunctionTool(func=get_best_usage_windows)

root_agent = Agent(
    name="elec_wise_agent",
    model="gemini-2.0-flash",
    description="Agent that helps optimize electricity usage based on grid carbon intensity",
    instruction="""You are ElecWise, an energy advisor helping users make eco-friendly electricity decisions.

Your tools:
1. get_carbon_intensity - Current grid carbon intensity for a zone
2. get_carbon_intencity_forecast - 24-hour carbon forecast  
3. get_best_usage_windows - Find optimal times with emission comparisons (current vs best vs best daytime)

When responding, always show:
- Current carbon intensity (gCO2/kWh) if running now
- Best overall time and its carbon intensity
- Best daytime option (if different) and its carbon intensity
- Percentage savings compared to running now

Zone codes: 'IE'=Ireland, 'GB'=Britain, 'DE'=Germany, 'FR'=France""",
    tools=[carbon_intensity_tool, carbon_forecast_tool, best_windows_tool]
)

In [27]:
async def run_agent(user_query: str):
    """Run the agent with InMemoryRunner and return response."""
    runner = InMemoryRunner(agent=root_agent, app_name="elec_wise_agent")
    session = await runner.session_service.create_session(
        app_name="elec_wise_agent",
        user_id="user1"
    )
    
    content = types.Content(
        role="user",
        parts=[types.Part(text=user_query)]
    )
    
    final_response = ""
    for event in runner.run(
        user_id="user1",
        session_id=session.id,
        new_message=content
    ):
        if event.is_final_response() and event.content and event.content.parts:
            final_response = event.content.parts[0].text
    
    return final_response

In [28]:
response = await run_agent("When is the best time to run my washing machine in Ireland today?")
print(response)

App name mismatch detected. The runner is configured with app name "elec_wise_agent", but the root agent was loaded from "C:\Users\rbsul\Documents\GitHub\elec-wise\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".


Here's the best time to run your washing machine in Ireland today:

*   **Current carbon intensity (now):** 300 gCO2/kWh
*   **Best overall time:** 5:00-6:00 with a carbon intensity of 172.5 gCO2/kWh (42.5% savings vs. now)
*   **Best daytime option:** 10:00-11:00 with a carbon intensity of 207.5 gCO2/kWh (30.8% savings vs. now)
